In [83]:
import socket
import re
import base64
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA256
from Crypto.Cipher import ChaCha20
from Crypto.PublicKey import RSA

LABEL1 = b'\x01'
LABEL2 = b'\x02'
LABEL3 = b'\x03'

NONCE_LEN = 128 // 8

def decrypt_flag(enc_flag_b64, nonceA, nonceB):
  enc_flag = base64.b64decode(enc_flag_b64)
  nonce = enc_flag[:8]
  ciphertext = enc_flag[8:]
  cipher = ChaCha20.new(key=nonceA + nonceB, nonce=nonce)
  flag = cipher.decrypt(ciphertext)
  return flag.decode('ascii')

def send_message(socket, message):
    socket.sendall(message.encode())
    response = b''
    while True:
        chunk = socket.recv(4096)
        if not chunk:
            break
        response += chunk
        if len(chunk) < 4096:
            break
    response = response.decode()
    return response

def receive_message(socket):
    chunks = []
    while True:
      chunk = socket.recv(4096)
      if not chunk:
        break
      chunks.append(chunk)
      if len(chunk) < 4096:
        break
    response = b''.join(chunks).decode()
    return response

def get_pubkey(message):
  match = re.search(r'\(e, n\) = \((\d+), (\d+)\)', message)
  if match:
    e = int(match.group(1))
    n = int(match.group(2))
    return e, n
  else:
    return None, None
  

def get_message(message):
  match = re.search(r'Here is message \d+:\s*([^\n]+)', message)
  if match:
    return match.group(1)
  else:
    return None
  
def get_flag(message):
  match = re.search(r'Here is the flag:\s*([^\n]+)', message)
  if match:
    return match.group(1)
  else:
    return None

In [84]:

# Generate a new RSA key pair
key = RSA.generate(2048 + 1024)
eve_sk = key
eve_pk = key.publickey()

# Extract (e, n) for the public key
e_eve = eve_pk.e
n_eve = eve_pk.n

print("New private key d:", eve_sk.d)
print("New public key (e, n):", (e_eve, n_eve))
print("Length of n:", len(str(n_eve)))
print("Length of e:", len(str(e_eve)))

New private key d: 59707877618404885915029551234281291624478041261440877337662080902156037809936675394461062452211363300357512837914509526956323692602867563574264427161527340884124896960053484844850163225639361282937163182840992640103132711851305541855275120049287657242189734937968710758643834874085637187955816575185089581824811779736590954787070477864766563675271320807424582414742422457530672867487364772306492556851248023174939716330684310344138205325839430248108404091024136533751509360754897164055260071915936274480042072395329767452497517446313313273654852288623226484183206904758741158954396195739172665757948692876024749040146417938062232522894850210624660900214636619395281731147400734136804880334978914235043317787185750033533496629914335808496416271702906989292637200037567941053655080119616571479717627279909539145463362891212722934915923000892122382656809821097494732568662482030796803567741415286085213505277257996415159091389
New public key (e, n): (65537, 33288602088280740180461860

In [85]:
alice_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
alice_socket.connect(("alice-chall.trojanc.tf", 5000))

response_alice = receive_message(alice_socket)
print("Alice's message:", response_alice)
e_alice, n_alice = get_pubkey(response_alice)
print("Alice's public key (e, n):", (e_alice, n_alice))
print("Length of n:", len(str(n_alice)))
print("Length of e:", len(str(e_alice)))

Alice's message: Hello, I am Alice, here is my public key: (e, n) = (65537, 516165688551264157696678017027953691200882154195033846605644533303630048284137174264513692702283032698118469473501514303809556286424207957055678034257786506888872702833390297252885339138383305924334576520065290866474650778794840417481500607056364194410542935756669704890259495646681350375622271600821580425508979728583337946069597088435448822477593484989926844051032563741386261374679320538527917266243913941794203253872494577188297868445727197775083756408153120961963009981184613501753755529915232551411547259276193116345979564063765730128235290924666349187009326230805567315294600038312713872139578894879536442853802113522247594760707885778484317213053180619073389523328653870091987658361018719867717675850638120248687047205824378749206685973094040227850954885288847360101386088859325748724307489496234909819833841644943207099946386693547140082280174625881308979923291822997032351750248905630103771334158554661649103416073

In [86]:
bob_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
bob_socket.connect(("bob-chall.trojanc.tf", 5000))

response_bob = receive_message(bob_socket)
print("Bob's message:", response_bob)
e_bob, n_bob = get_pubkey(response_bob)
print("Bob's public key (e, n):", (e_bob, n_bob))

Bob's message: Hello, I am Bob, here is my public key: (e, n) = (65537, 3368571081442952949585615985860628486223488607047576108935583296034904626981874065519113682057883471590363919785148743976643752364751186817413918655233957819800641448578978675380622570320570754477625583387542152525987312312454408884491865687216606274055920498296433909875239500500716482141733133389911514120220616083269385553286932156135865486813351454447581866279936043528928199584300001255197725370838900154424529061536576297585539634000508282347883560410823597197996822765028001681156652599736004347511516694476005536628728265621917143707466363647223716111584522613067543633400672794221600598543911647396320247372942051672684249249789467542019907568976467007594967136817023346356029767229792121947286576874741931473992909092339049489544615262689183331089235604474072800912295172930928290508568698428328466898814512513541650087329798505259352309392454677737860101235744948015068871874814113100721526585222345252980847127871)
P

In [87]:
messages = [
  "1\n",
  "Eve\n",
  f"{e_eve}\n",
  f"{n_eve}\n",
]

for message in messages:
  print("Sending message:", message.strip())
  print(send_message(alice_socket, message))

Sending message: 1
Please input your name: 
Sending message: Eve
Please input your public key exponent: 
Sending message: 65537
Please input your public key modulus n: 
Sending message: 3328860208828074018046186047844400688382320195789919845238927942224245214759523517930067758341621536891135958195153731066045585573895475554203800734057862475136446934981731379223262566838559609017314218216801475673703963025605283961351906033747482086500543308064360184593144199185836158559813144098600779178266856323774531181522958662546326060047857551472722124810696844405944455735022905217057084388226492296738429765345859333920702307477276682407724780019947971333065727111182477954932106130118790393608780357046956504133156615539620103144630446129990749167020056632205860516761107467517933692544487777503778782904035013395872270931566373828133520425819175804548144657865972673571197357678589887824958570649378917352779072853045484614871673989944893465952315434889699670231073587881341730730774650196639701155539

In [88]:
send_message(alice_socket, "2\n")
send_message(bob_socket, "2\n")

mess_1 = get_message(send_message(alice_socket, "Eve\n"))
print("Alice's message (base64):", mess_1)

# Decode the base64-encoded ciphertext
cipher1 = PKCS1_OAEP.new(eve_sk, hashAlgo=SHA256, label=LABEL1)
decrypted_eve_1 = cipher1.decrypt(base64.decodebytes(bytes(mess_1, 'ascii')))
nonce_alice = decrypted_eve_1[:NONCE_LEN]
print("Nonce from Alice:", nonce_alice)

cipher_bob_1 = PKCS1_OAEP.new(RSA.construct((n_bob, e_bob)), hashAlgo=SHA256, label=LABEL1)
encrypted_alice_1 = cipher_bob_1.encrypt(nonce_alice + bytes("Alice", 'ascii'))

mess_2 = get_message(send_message(bob_socket, base64.b64encode(encrypted_alice_1).decode() + "\n"))
print("Bob's message:", mess_2)

mess_3 = get_message(send_message(alice_socket, mess_2 + "\n"))
print("Alice's message:", mess_3)

# Decode the base64-encoded ciphertext
cipher3 = PKCS1_OAEP.new(eve_sk, hashAlgo=SHA256, label=LABEL3)
decrypted_eve_3 = cipher3.decrypt(base64.decodebytes(bytes(mess_3, 'ascii')))
nonce_bob = decrypted_eve_3[:NONCE_LEN]
print("Nonce from Bob:", nonce_bob)

cipher_bob_3 = PKCS1_OAEP.new(RSA.construct((n_bob, e_bob)), hashAlgo=SHA256, label=LABEL3)
encrypted_alice_3 = cipher_bob_3.encrypt(nonce_bob)

mess_4 = send_message(bob_socket, base64.b64encode(encrypted_alice_3).decode() + "\n")
print("Bob's message:", mess_4)
flag = get_flag(mess_4)
print("Flag (base64):", flag)

Alice's message (base64): QMy+C20jvA1dwblUGv3roXcRIgDNu9sTUMSpb74cH08CeVuB0zRNsadMBX0b04I+xC15iHLvoh+2MZw3qMMwAhpNxZoNPqSaH5ostAJaYMa61SJgMdGz+vMtxiyaH0UCKRTEsf264Q5YLj5Nct2X2fL2asEy3LS8WciFdoDvWGwXpE8fUov/rEyb6Gn9NiFqCS3MCLjuCU/6q4IUpeqg2+Woxzb2lnVkTkGt549wg4aknRt8XXe74sGCY+Mdl5COXfJV2IasKGUkkIwDdFllLfEtFXFgx7/hcK/qqcfv4wLwxuyw/nzxN/jL714K0kxAEwRHJ6Fq0eXCgcHo4Zm46d/uNVcS3WGrdNgcDFxntwp6ArYQhQlbzlZBz2tJd5kW3C77TSD5/cI5ZRJrFZSfL8HSeH1OFX7cRPti85hgzRAnulIMI0Lw91sfla2ofyehNzRkxVoOCzqWLM8WwdhbNG7F10v0qqyz9stErEwdfWbcBTwE0deApqwgGU/ADxeM
Nonce from Alice: b'\x10\x9eK}z\xae\xae;_|\xa7-\xefq\x98}'
Bob's message: DTnThMnGyiXr1fayJrUNrzKpAD9yQtDwh7Ulwm5V6lZPNeUxqT/kpgT9nVNS9CJYThZWSzBP5x+2QEO+UegW+zs+0DHG7/EPa1nCyhWfuUcG4EEWHAyd0pmUpa25zhBmvNqDP976EO7nNyUQEiRlua9qnPEiEWhken7IKjlLQVzyn9Gr8qTK7bWDDzZ+p2m/VSKkETcwldqcH2XQ+gx3rabWf6m42oJQVDEoHw4DAXs6KUYSzDayaD7AkuSa/eT5hdjFeyiAq9N1IKAOWiymeVrORVh0x6PuDHvBMK3REaIzT6ES7VgpABto5eZBq3J7QVxWv2ARgpusxYhbgVqMcnthLwi7VkJ3ENSvk59crrso+wFK2J5Oqoa+Z0gzW358YWa

In [89]:
decrypt_flag(flag, nonce_alice, nonce_bob)

'Trojan{Sec_pr0t0c0ls_are_3_line_pr0grams_that_pe0ple_still_m@n@ge_to_get_wr0ng}'